In [ ]:
pip install pymysql

In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from IPython.display import JSON
import datetime
import sqlalchemy
import pymysql
import numpy as np

In [9]:
# To use the code you need to import a .py file with your keys 

import API_key

API_key1 = API_key.API_key1
X_RapidAPI_Key = API_key.X_RapidAPI_Key
sql_key = API_key.SQL_key
password_aws = API_key.password_aws

## PART I: WEB SCRAPING

## MANUALLY COLLECTING DATA ON THE LARGEST EUROPEAN CITIES FROM WIKIPEDIA

In [11]:
# Creating a function that loops through the scraped data saved in 'rows' 
# and appends it to the empty dictionary 'cities_dict' which is then converted 
# to a dataframe (cities_df) for futher use

def cities_func(): 
    
    cities_dict = {
    'city':[],
    'country':[],
    'population':[],
    'lat':[],
    'lon':[]
    }
    
# STEP 1: Saving the scraped data to a JSON
    url = "https://en.wikipedia.org/wiki/List_of_European_cities_by_population_within_city_limits"
    response = requests.get(url)
    response.status_code
    soup = BeautifulSoup(response.content)
    rows = soup.select("table.wikitable tr")
    
    # Run a loop through the scraped content ('rows') and append selected data points to empty dictionary
    for i, row in enumerate(rows):
        if(i == 0 ):
            continue
        cities_dict['city'].append(row.select('td')[1].select('a')[0].get_text().strip())
        cities_dict['country'].append(row.select('td')[2].select('a')[0].get_text().strip())
        cities_dict['population'].append(row.select('td')[3].select('span')[0].get_text())
        cities_dict['lat'].append(row.select('td')[7].select('span.geo-dec')[0].get_text().split()[0])#.strip('°N'))
        cities_dict['lon'].append(row.select('td')[7].select('span.geo-dec')[0].get_text().split()[1])#.strip('°E,°W'))
    # Converting dictionary to dataframe
    cities_df = pd.DataFrame(cities_dict)
    
# STEP 2 Cleaning the data:
# The columns latitude, longitude, and population contain string no longer required
    cities_df[['lat_','temp']] = cities_df['lat'].str.split('°',expand=True)
    cities_df['lat_'] = pd.to_numeric(cities_df['lat_'], errors='coerce')
    cities_df['lat'] = cities_df.apply(lambda x: x['lat_'] * (-1) if x['temp'] == 'S' else x['lat_'], axis=1)
    cities_df[['lon_','temp2']] = cities_df['lon'].str.split('°',expand=True)
    cities_df['lon_'] = pd.to_numeric(cities_df['lon_'], errors='coerce')
    cities_df['lon'] = cities_df.apply(lambda x: x['lon_'] * (-1) if x['temp2'] == 'W' else x['lon_'], axis=1)
    cities_df.drop(['lat_','temp','lon_','temp2'],axis=1,inplace=True)
    cities_df['lat'] = cities_df['lat'].round(decimals = 2)
    cities_df['lon'] = cities_df['lon'].round(decimals = 2)
   
    cities_df['population'] = pd.to_numeric(cities_df['population'].str.replace(',',''))
    return cities_df

#Calling the function
cities_df = pd.DataFrame(cities_func())
cities_df

,city,country,population,lat,lon
0,Istanbul,Turkey,15840900,41.01,28.96
1,Moscow,Russia,12632409,55.75,37.62
2,London,United Kingdom,8799800,51.51,-0.13
3,Saint Petersburg,Russia,5376672,59.95,30.30
4,Berlin,Germany,3677472,52.52,13.38
5,Madrid,Spain,3305408,40.38,-3.72
6,Kyiv,Ukraine,2962180,50.45,30.52
7,Rome,Italy,2749031,41.90,12.50
8,Baku,Azerbaijan,2303100,40.40,49.90
9,Bucharest,Romania,2161347,44.43,26.10


In [ ]:
# Step 3: Storing cities_df in mySQL for local use and in AWS 

In [5]:
#local
host = '127.0.0.1'
port = '3306'
schema = 'gans'
user = 'root'
password = sql_key
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [6]:
#AWS
host_aws = 'your.endpoint.eu-central-1.rds.amazonaws.com'
port_aws = '3306'
schema_aws = 'gans'
user_aws = 'admin'
pass_aws = password_aws
con_aws = f'mysql+pymysql://{user_aws}:{pass_aws}@{host_aws}:{port_aws}/{schema_aws}'

In [ ]:
# Pushing cities_df dataframe to mySQL database
# syntax: DataFrame.to_sql(name, con, schema=None, if_exists='fail', index=True, index_label=None, chunksize=None, dtype=None, method=None)

In [ ]:
# to AWS database
cities_df.to_sql('cities', con=con_aws, if_exists='append', index=False)

In [ ]:
# to local database
cities_df.to_sql('cities', con=con_aws, if_exists='append', index=False)

In [ ]:
# Read database from mySQL
#Syntax: pandas.read_sql(sql, con, index_col=None, coerce_float=True, params=None, parse_dates=None, columns=None, chunksize=None)

In [13]:
cities_df = pd.read_sql('cities', con=con_aws)

## PART II: AIRPORTS

## IMPORTING AIRPORT DATA FROM CSV FILE

In [8]:
#IMPORT THE AIPORTS FROM A CSV FILE
airports_from_csv = pd.read_csv('/home/jh23/Documents/wissenschaft/data_science/projects/05 data engineering/airports.csv')

In [11]:
#airports table is edited to only contain relevant and correctly named columns and only 2 airports to reduce the number of API calls on RapidAPI 
airports_brief = airports_from_csv[['name','municipality','gps_code']]
airports_brief  = airports_brief.dropna()
airports_brief['icao'] = airports_brief['gps_code']
airports_brief['city'] = airports_brief['municipality']
airports_brief['airport'] = airports_brief['name']
airports_brief = airports_brief[['icao','airport','city']]
airports_brief = pd.merge(airports_brief, cities_df[['city','city_id']], on='city', how='left').dropna()
airports_brief = airports_brief[['city_id','icao','airport','city']]
airports = airports_brief.loc[airports_brief['icao'].isin(['EDDB','EGLL'])]

,icao,airport,city
0,00A,Total Rf Heliport,Bensalem
1,00AA,Aero B Ranch Airport,Leoti
2,00AK,Lowell Field,Anchor Point
3,00AL,Epps Airpark,Harvest
5,00AS,Fulton Airport,Alex
...,...,...,...
68326,ZYYK,Yingkou Lanqi Airport,"Laobian, Yingkou"
68327,ZYYY,Shenyang Dongta Airport,"Dadong, Shenyang"
68330,87TX,Fainting Goat Airport,Blum
68331,ZZZW,Scandium City Heliport,(Old) Scandium City


In [21]:
# Sending to AWS database
airports.to_sql('airports', con=con_aws, if_exists='append', index=False)

2

In [22]:
# Reading from AWS database
airports = pd.read_sql('airports', con=con_aws)

,city_id,icao,airport,city
0,5,EDDB,Berlin Brandenburg Airport,Berlin
1,3,EGLL,London Heathrow Airport,London


## PART III: WEATHER

## CALLING THE OPENWEATHER API AND SAVE JSON DATA TO DATAFRAME

In [ ]:
# Step I: Saving the API data in a JSON

lat = cities_df['lat'][0]
lon = cities_df['lon'][0]
API_key = API_key 
url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key}&units=metric'
r = requests.get(url)
weather = r.json()

In [ ]:
# STEP 2: Locating the relevant data points inside the weather json.

In [ ]:
#weather['list'][0]['dt_txt'] #timestamp

In [ ]:
#weather['list'][0]['main']['temp'] #temperature

In [ ]:
#weather['list'][0]['main']['feels_like'] #felt temperature

In [ ]:
#weather['city']['id'] #unique identifier

In [ ]:
#weather['city']['name'] #unique identifier

In [ ]:
#weather['list'][0]['wind']['speed'] #wind speed

In [ ]:
#weather['list'][0]['pop'] #probability of rain

In [ ]:
#weather['list'][0]['wind']['speed'] #wind speed

In [ ]:
#weather['list'][0]['weather'][0]['description'] #weather description

In [36]:
#cities_df.iloc[1]['lat']

55.75

In [26]:
def weather_func(cities_df):
    
    weather_dict = {
    'city_id':[],
    'city':[],
    'description':[],
    'temperature':[],
    'felt_temperature':[],
    'wind_speed':[],
    'rain_probability':[],
    'timestamp':[]  
    }
    for i, w in enumerate(cities_df['city']):
        lat = cities_df.iloc[i]['lat']
        lon = cities_df.iloc[i]['lon']
        API_key= API_key1
        url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key}&units=metric'
        r = requests.get(url)
        weather = r.json()
        for time in weather['list']:
            weather_dict['city_id'].append(cities_df.iloc[i]['city_id'])
            weather_dict['city'].append(cities_df.iloc[i]['city'])
            weather_dict['description'].append(time['weather'][0]['description']) #weather description
            weather_dict['temperature'].append(time['main']['temp']) #temperature
            weather_dict['felt_temperature'].append(time['main']['feels_like']) #felt temperature
            weather_dict['wind_speed'].append(time['wind']['speed']) #wind speed
            weather_dict['rain_probability'].append(time['pop']) #probability of rain
            weather_dict['timestamp'].append(time['dt_txt']) #timestamp
    weather_df = pd.DataFrame(weather_func(cities_df))
    weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
    return weather_df

weather_func(cities_df)
weather_df


In [ ]:
# to SQL
weather_df.to_sql('weather', con=con, if_exists='append', index=False)

In [ ]:
# from SQL
weather_sql = pd.read_sql('weather', con=con)
weather_sql

## PART IV: FLIGHTS 

## COLLECTING FLIGHT DATA FROM RAPIDAPI 

In [ ]:
# API call for arrivals

today = datetime.date.today()
tomorrow = today + datetime.timedelta(days=1)
icao_list = airports_sql['icao'][0:2].to_list() #only 2 airports are used because of API call number constraint
times = [["00:00","11:59"],["12:00","23:59"]]

def arrivals_func(tomorrow,icao_list,times,X_RapidAPI_Key):
    arrivals_dict = {
    'icao':[],
    'scheduledTimeLocal':[],
    'departure_icao':[]      
    }
        
    for time in times: #iteration is necessary because of 12h API constraint
        
        for icao in icao_list:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
            headers = {
            "X-RapidAPI-Key": f"{X_RapidAPI_Key}",
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }
            response = requests.request("GET", url, headers=headers, params=querystring)
            arrival_response = response.json()
            for flight in arrival_response['arrivals']:
                try:
                    arrivals_dict['icao'].append(icao)
                except:
                    arrivals_dict['icao'].append('unknown')
                try:
                    arrivals_dict['scheduledTimeLocal'].append(flight['arrival']['scheduledTimeLocal'])
                except:
                    arrivals_dict['scheduledTimeLocal'].append('unknown')
                try:
                    arrivals_dict['departure_icao'].append(flight['departure']['airport']['icao'])
                except:
                    arrivals_dict['departure_icao'].append('unknown')
                             
                #print(arrivals_dict)
                #print('=========================')
    return arrivals_dict

arrivals_df = pd.DataFrame(arrivals_func(tomorrow,icao_list,times,X_RapidAPI_Key))
arrivals_df
                     

# Lambda function to use on AWS

In [ ]:
import json
import requests
import pandas as pd
import sqlalchemy
import pymysql
import API_key
import datetime



def lambda_handler(event, context):
    # Connect to database
    password_aws = API_key.password_aws
    host_aws = 'your.endpoint.eu-central-1.rds.amazonaws.com'
    port_aws = '3306'
    schema_aws = 'gans'
    user_aws = 'admin'
    con_aws = f'mysql+pymysql://{user_aws}:{password_aws}@{host_aws}:{port_aws}/{schema_aws}'
    
    #Read cities and airports from existing SQL tables
    cities_df = pd.read_sql('cities', con=con_aws)
    airports_df = pd.read_sql('airports', con=con_aws)
    
    #Calling weather function to access OpenWeather API
    weather_df = get_weather(cities_df)
    weather_df.to_sql('weather', con=con_aws, if_exists='append', index=False)
    
    #Calling arrivals function to access RapidAPI AeroDataBox
    arrivals_df = get_flights(airports_df)
    arrivals_df.to_sql('arrivals', con=con_aws, if_exists='append', index=False)
    
def get_weather(cities_df):
    
    weather_dict = {
    'city_id':[],
    'city':[],
    'description':[],
    'temperature':[],
    'felt_temperature':[],
    'wind_speed':[],
    'rain_probability':[],
    'timestamp':[]  
    }
    for i, w in enumerate(cities_df['city']):
        lat = cities_df.iloc[i]['lat']
        lon = cities_df.iloc[i]['lon']
        API_key1 = API_key.API_key1
        url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key1}&units=metric'
        r = requests.get(url)
        weather = r.json()
        for time in weather['list']:
            weather_dict['city_id'].append(cities_df.iloc[i]['city_id'])
            weather_dict['city'].append(cities_df.iloc[i]['city'])
            weather_dict['description'].append(time['weather'][0]['description']) #weather description
            weather_dict['temperature'].append(time['main']['temp']) #temperature
            weather_dict['felt_temperature'].append(time['main']['feels_like']) #felt temperature
            weather_dict['wind_speed'].append(time['wind']['speed']) #wind speed
            weather_dict['rain_probability'].append(time['pop']) #probability of rain
            weather_dict['timestamp'].append(time['dt_txt']) #timestamp
    weather_df = pd.DataFrame(weather_dict)
    weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
    return weather_df 
    

def get_flights(airports_df):
    today = datetime.date.today()
    tomorrow = today + datetime.timedelta(days=1)
    icao_list = airports_df['icao'].to_list()
    times = [["00:00","11:59"],["12:00","23:59"]]
    
    arrivals_dict = {
    'icao':[],
    'time':[],
    'departure_icao':[]      
    }
        
    for time in times:
        
        for icao in icao_list:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
            headers = {
            "X-RapidAPI-Key": "your_key",
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }
            response = requests.request("GET", url, headers=headers, params=querystring)
            arrival_response = response.json()
            for flight in arrival_response['arrivals']:
                try:
                    arrivals_dict['icao'].append(icao)
                except:
                    arrivals_dict['icao'].append('unknown')                
                try:
                    arrivals_dict['departure_icao'].append(flight['departure']['airport']['icao'])
                except:
                    arrivals_dict['departure_icao'].append('unknown')
                try:
                    arrivals_dict['time'].append(flight['arrival']['scheduledTimeUtc'])
                except:
                    arrivals_dict['time'].append(pd.NaT)
    
    arrivals_df = pd.DataFrame(arrivals_dict)                          
    arrivals_df['time'] = pd.to_datetime(arrivals_df['time']) 
    return arrivals_df
    
    
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }
